In [46]:
import pickle
import matplotlib.pyplot as plt # creating visualizations
import numpy as np # basic math and random numbers
import torch # package for building functions with learnable parameters
import torch.nn as nn # prebuilt functions specific to neural networks
from torch.autograd import Variable # storing data while learning
#from disp import ADE,FDE,prediction_displacement,ADE_double_coordinates,FDE_double_coordinates,prediction_displacement_double


In [48]:
import numpy as np
import pandas as pd
# Predict error
# Average Displacement error

def ADE(pred,truth): 
    counter=0
    sum=0
    for i in range(len(pred)):
        half=int(len(pred[i])/2)
        for j in range (half):

            a = np.array((pred[i][j] , pred[i][half+j]))
            b = np.array((truth.iloc[i][j] , truth.iloc[i][half+j]))

            dist = np.linalg.norm(a-b)
            sum+=dist
            counter+=1
            #print("Distance between",a," and ",b," is: ",dist)

    return (sum/counter)
def prediction_displacement_double(pred): 
          
        sum=0
        counter=0        
        pred=np.array(pred)
        last_index= (len(pred[0])-1)
        
        for i in range(len(pred)):
                
                    a = np.array((pred[i][0][0] , pred[i][0][1]))
                    b = np.array((pred[i][last_index][0] , pred[i][last_index][1]))

                    dist = np.linalg.norm(a-b)
                    sum+=dist
                    counter+=1
        return (sum/counter)

def prediction_displacement(pred): 
    counter=0
    sum=0
    pred=np.array(pred)
    for i in range(len(pred)):
        half=int(len(pred[i])/2)
        last=(len(pred[i]) - 1)

        a = np.array((pred[i][0] , pred[i][half]))
        b = np.array((pred[i][half-1] , pred[i][last]))

        dist = np.linalg.norm(a-b)
        sum+=dist
        counter+=1
        #print("FDE Distance between",a," and ",b," is: ",dist)
            
    return (sum/counter)
def FDE(pred,truth): 
    counter=0
    sum=0
    for i in range(len(pred)):
        half=int(len(pred[i])/2)
        last=(len(pred[i]) - 1)

        a = np.array((pred[i][half-1] , pred[i][last]))
        b = np.array((truth.iloc[i][half-1] , truth.iloc[i][last]))

        dist = np.linalg.norm(a-b)
        sum+=dist
        counter+=1
        #print("FDE Distance between",a," and ",b," is: ",dist)
            
    return (sum/counter)


def FDE_double_coordinates(pred,truth):
          
        sum=0
        counter=0
        
        pred=np.array(pred)
        truth=np.array(truth)
        last_index= (len(pred[0])-1)

        for i in range(len(pred)):
                
            a = np.array((pred[i][last_index][0] , pred[i][last_index][1]))
            b = np.array((truth[i][last_index][0] , truth[i][last_index][1]))

            dist = np.linalg.norm(a-b)
            sum+=dist
            counter+=1

                # for j in range (len(pred[i])):
                #    pred_x.append(pred[i][j][0])  
                #    pred_y.append(pred[i][j][1]) 

                #    truth_x.append(truth[i][j][0])  
                #    truth_y.append(truth[i][j][1])
        return (sum/counter)
def ADE_double_coordinates(pred,truth):
          
        sum=0
        counter=0
        
        pred=np.array(pred)
        truth=np.array(truth)

        for i in range(len(pred)):
                
                for j in range (len(pred[i])):

                    a = np.array((pred[i][j][0] , pred[i][j][1]))
                    b = np.array((truth[i][j][0] , truth[i][j][1]))

                    dist = np.linalg.norm(a-b)
                    sum+=dist
                    counter+=1
        return (sum/counter)

In [2]:
def load_data():
    with open('train_input.pickle', 'rb') as data:
     train_input = pickle.load(data)
    with open('validate_input.pickle', 'rb') as data:
         validate_input= pickle.load(data)
    with open('test_input.pickle', 'rb') as data:
         test_input = pickle.load(data)
    with open('train_target.pickle', 'rb') as data:
          train_target = pickle.load(data)
    with open('validate_target.pickle', 'rb') as data:
          validate_target = pickle.load(data)
    with open('test_target.pickle', 'rb') as data:
         test_target = pickle.load(data)
    return torch.tensor(train_input,dtype=torch.float32), torch.tensor(validate_input,dtype=torch.float32), torch.tensor(test_input,dtype=torch.float32), torch.tensor(train_target,dtype=torch.float32), torch.tensor(validate_target,dtype=torch.float32), torch.tensor(test_target,dtype=torch.float32)

In [33]:
train_input,validate_input,test_input,train_target,validate_target,test_target=load_data()
print("Train shape: " ,train_target.shape)

Train shape:  torch.Size([5099, 8, 2])


In [34]:
class LinearNetwork(nn.Module):
    def __init__(self):
        super(LinearNetwork, self).__init__()
        self.output_shape = (8,2)
        self.flatten = nn.Flatten()
        self.unflatten = nn.Unflatten(1,self.output_shape)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(8*2, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 8*2),
        )

    def forward(self, x):
        x = self.flatten(x)
        # print(x[0])
        logits = self.linear_relu_stack(x)
        out = self.unflatten(logits)
        # print(out[0])
        return out

In [35]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [36]:
model = LinearNetwork().to(device)
print(model)

LinearNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (unflatten): Unflatten(dim=1, unflattened_size=(8, 2))
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=16, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=16, bias=True)
  )
)


In [37]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.RMSprop(model.parameters())

In [85]:
train_input. requires_grad = True
train_input. requires_grad 

True

In [39]:
y_pred = model(train_input)
y_pred.shape
print (y_pred[0])
sq = torch.flatten(y_pred,1)
sq2 = torch.flatten(train_input,1)
print("///////////")
print(sq[0])
print(sq2[0])
# temp = nn.Flatten(train_input)
# train_input.shape,temp.shape,y_pred[0]


tensor([[-1.6634,  0.5903],
        [ 0.4768, -0.8592],
        [ 0.0575, -0.2041],
        [ 0.4909, -1.4022],
        [-0.4778,  1.3517],
        [ 0.2441, -0.9381],
        [-1.7007,  0.1728],
        [-0.5561, -0.0388]], grad_fn=<SelectBackward0>)
///////////
tensor([-1.6634,  0.5903,  0.4768, -0.8592,  0.0575, -0.2041,  0.4909, -1.4022,
        -0.4778,  1.3517,  0.2441, -0.9381, -1.7007,  0.1728, -0.5561, -0.0388],
       grad_fn=<SelectBackward0>)
tensor([14.9352,  5.3071, 14.4947,  5.3293, 14.0542,  5.3515, 13.6137,  5.3737,
        13.1726,  5.3937, 12.7264,  5.3937, 12.2802,  5.3937, 11.8340,  5.3937],
       grad_fn=<SelectBackward0>)


In [40]:
def train():
    for epoch in range(3000):
        y_pred = model(train_input) # make a prediction
        loss = loss_fn(y_pred,train_target) # compute the loss
        optimizer.zero_grad() # prepare the optimizer
        loss.backward() # compute the contribution of each parameter to the loss
        optimizer.step() # modify the parameters

        if epoch % 300 == 0:
            print(epoch, loss.data)

train()

0 tensor(47.2369)
300 tensor(0.6566)
600 tensor(0.7877)
900 tensor(0.6171)
1200 tensor(0.9639)
1500 tensor(0.7391)
1800 tensor(2.4894)
2100 tensor(0.8386)
2400 tensor(1.0475)
2700 tensor(0.4876)


In [56]:
with torch.no_grad():
    test_pred = model(test_input)
    train_pred = model(train_input)
    loss = loss_fn(test_target, test_pred)
    print(loss.data)

tensor(1.2341)


In [58]:
def show_error_double(prediction_train,prediction_test,train_target,test_target):
                # # Show error rate
        print("AVERAGE DISTANCE BETWEEN FIRST AND LAST POINT Train: ",prediction_displacement_double(train_target))
        print("AVERAGE DISTANCE BETWEEN FIRST AND LAST POINT Test: ",prediction_displacement_double(test_target))
             
        #average_displacement_error
        print("ADE ERROR RATE TEST: ", ADE_double_coordinates(prediction_test,test_target))
        #average_displacement_error
        print("ADE ERROR RATE TRAIN: ", ADE_double_coordinates(prediction_train,train_target))
        print("//////////////////////////////////////////")
        #Final_displacement_error
        print("FDE ERROR RATE TEST: ", FDE_double_coordinates(prediction_test,test_target))
        print("FDE ERROR RATE TRAIN: ", FDE_double_coordinates(prediction_train,train_target))


In [61]:
show_error_double(train_pred,test_pred,train_target,test_target)

AVERAGE DISTANCE BETWEEN FIRST AND LAST POINT Train:  1.7491152588806866
AVERAGE DISTANCE BETWEEN FIRST AND LAST POINT Test:  2.7941778349259665
ADE ERROR RATE TEST:  1.424682301002654
ADE ERROR RATE TRAIN:  1.5516014411372374
//////////////////////////////////////////
FDE ERROR RATE TEST:  1.419016819525893
FDE ERROR RATE TRAIN:  1.5553257257172959


In [105]:
class LSTM(nn.Module):
    def __init__(self, hidden_layers=12):
        super(LSTM, self).__init__()
        self.hidden_layers = hidden_layers
        # lstm1, lstm2, linear are all layers in the network
        self.lstm1 = nn.LSTMCell(1, self.hidden_layers)
        self.lstm2 = nn.LSTMCell(self.hidden_layers, self.hidden_layers)
        self.linear = nn.Linear(self.hidden_layers, 1)
        
    def forward(self, y, future_preds=0):
        outputs, n_samples = [], y.size(0)
        print('n_samples',n_samples)
        h_t = torch.zeros(n_samples, self.hidden_layers, dtype=torch.float32)
        c_t = torch.zeros(n_samples, self.hidden_layers, dtype=torch.float32)
        h_t2 = torch.zeros(n_samples, self.hidden_layers, dtype=torch.float32)
        c_t2 = torch.zeros(n_samples, self.hidden_layers, dtype=torch.float32)
        
        for time_step in y.split(1, dim=1):
            # N, 1
            # input_t = time_step.s
            print("time_step",time_step.shape)
            h_t, c_t = self.lstm1(time_step, (h_t, c_t)) # initial hidden and cell states
            
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2)) # new hidden and cell states
            output = self.linear(h_t2) # output from the last FC layer
            outputs.append(output)
            
        for i in range(future_preds):
            # this only generates future predictions if we pass in future_preds>0
            # mirrors the code above, using last output/prediction as input
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs.append(output)
        # transform list to tensor    
        outputs = torch.cat(outputs, dim=1)
        return outputs

In [106]:
model_LSTM = LSTM()
criterion_LSTM = nn.MSELoss()
optimiser_LSTM = torch.optim.LBFGS(model_LSTM.parameters(), lr=0.08)

In [107]:

def training_loop(n_epochs, model, optimiser, loss_fn, 
                  train_input, train_target, test_input, test_target):
    for i in range(n_epochs):
        def closure():
            optimiser.zero_grad()
            out = model(train_input)
            print("okkokokok",out.shape)
            loss = loss_fn(out, train_target)
            loss.backward()
            return loss
        optimiser.step(closure)
        with torch.no_grad():
            future = 1000
            pred = model(test_input, future=future)
            # use all pred samples, but only go to 999
            loss = loss_fn(pred[:, :-future], test_target)
            y = pred.detach().numpy()
        # draw figures
        plt.figure(figsize=(12,6))
        plt.title(f"Step {i+1}")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        n = train_input.shape[1] # 999
        def draw(yi, colour):
            plt.plot(np.arange(n), yi[:n], colour, linewidth=2.0)
            plt.plot(np.arange(n, n+future), yi[n:], colour+":", linewidth=2.0)
        draw(y[0], 'r')
        draw(y[1], 'b')
        draw(y[2], 'g')
        plt.savefig("predict%d.png"%i, dpi=200)
        plt.close()
        # print the loss
        out = model(train_input)
        loss_print = loss_fn(out, train_target)
        print("Step: {}, Loss: {}".format(i, loss_print))

In [108]:
training_loop(n_epochs=64, model=model_LSTM, optimiser=optimiser_LSTM, loss_fn=criterion_LSTM, 
                  train_input=train_input, train_target=train_target, test_input=test_input, test_target=test_target)

time_step torch.Size([5099, 1, 2])


AssertionError: LSTMCell: Expected input to be 1-D or 2-D but received 3-D tensor

In [ ]:
rnn = nn.LSTMCell(8, 12) # (input_size, hidden_size)
input = torch.randn(2, 3, 8) # (time_steps, batch, input_size)
input = torch.randn(8, 2, 8) # (time_steps, batch, input_size)
hx = torch.randn(2, 12) # (batch, hidden_size)
cx = torch.randn(2, 12)
output = []

for i in range(input.size()[0]):
    hx, cx = rnn(input[i], (hx, cx))
    print(input[i].shape, hx.shape, cx.shape)
    output.append(hx)
output = torch.stack(output, dim=0)



torch.Size([2, 8]) torch.Size([2, 12]) torch.Size([2, 12])
torch.Size([2, 8]) torch.Size([2, 12]) torch.Size([2, 12])
torch.Size([2, 8]) torch.Size([2, 12]) torch.Size([2, 12])
torch.Size([2, 8]) torch.Size([2, 12]) torch.Size([2, 12])
torch.Size([2, 8]) torch.Size([2, 12]) torch.Size([2, 12])
torch.Size([2, 8]) torch.Size([2, 12]) torch.Size([2, 12])
torch.Size([2, 8]) torch.Size([2, 12]) torch.Size([2, 12])
torch.Size([2, 8]) torch.Size([2, 12]) torch.Size([2, 12])
